### What is Azure OpenAI Service?

Azure OpenAI Service provides REST API access to OpenAI's powerful language models including the GPT-3, Codex and Embeddings model series. In addition, the new GPT-4 and ChatGPT (gpt-35-turbo) model series have now reached general availability. These models can be easily adapted to your specific task including but not limited to content generation, summarization, semantic search, and natural language to code translation. Users can access the service through REST APIs, Python SDK, or our web-based interface in the Azure OpenAI Studio.

#### Azure OpenAI Access

Access is currently limited as we navigate high demand, upcoming product improvements, and [Microsoft’s commitment to responsible AI](https://www.microsoft.com/ai/responsible-ai?activetab=pivot1:primaryr6). For now, we're working with customers with an existing partnership with Microsoft, lower risk use cases, and those committed to incorporating mitigations.

More specific information is included in the application form. We appreciate your patience as we work to responsibly enable broader access to Azure OpenAI.

Apply [here](https://aka.ms/oaiapply) for access

#### Key concepts - Prompts & Completions

The completions endpoint is the core component of the API service. This API provides access to the model's text-in, text-out interface. Users simply need to provide an input prompt containing the English text command, and the model will generate a text completion.

Here's an example of a simple prompt and completion:

Prompt: """ count to 5 in a for loop """

Completion: for i in range(1, 6): print(i)

#### Key concepts - Tokens

Azure OpenAI processes text by breaking it down into tokens. Tokens can be words or just chunks of characters. For example, the word “hamburger” gets broken up into the tokens “ham”, “bur” and “ger”, while a short and common word like “pear” is a single token. Many tokens start with a whitespace, for example “ hello” and “ bye”.

The total number of tokens processed in a given request depends on the length of your input, output and request parameters. The quantity of tokens being processed will also affect your response latency and throughput for the models.

#### Key concepts - In-context learning
The models used by Azure OpenAI use natural language instructions and examples provided during the generation call to identify the task being asked and skill required. When you use this approach, the first part of the prompt includes natural language instructions and/or examples of the specific task desired. The model then completes the task by predicting the most probable next piece of text. This technique is known as "in-context" learning. These models aren't retrained during this step but instead give predictions based on the context you include in the prompt.

There are three main approaches for in-context learning: Few-shot, one-shot and zero-shot. These approaches vary based on the amount of task-specific data that is given to the model:

- Few-shot: In this case, a user includes several examples in the call prompt that demonstrate the expected answer format and content. The number of examples typically range from 0 to 100 depending on how many can fit in the maximum input length for a single prompt. Maximum input length can vary depending on the specific models you use. Few-shot learning enables a major reduction in the amount of task-specific data required for accurate predictions. This approach will typically perform less accurately than a fine-tuned model.

- One-shot: This case is the same as the few-shot approach except only one example is provided.

- Zero-shot: In this case, no examples are provided to the model and only the task request is provided.

#### Pre-requisites

An Azure subscription - [Create one for free](https://azure.microsoft.com/free/cognitive-services)

Access granted to Azure OpenAI in the desired Azure subscription

Currently, access to this service is granted only by application. You can apply for access to Azure OpenAI by completing the form [here](https://aka.ms/oai/access).

[Python 3.7.1 or later version](https://www.python.org/)

The following Python libraries: os, requests, json, openai

An Azure OpenAI Service resource with a model deployed. For more information about model deployment, see the [resource deployment guide](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource).

In [2]:
#Install the OpenAI Python client library with:
# %pip install openai

#### Retrieve key and endpoint
To successfully make a call against the Azure OpenAI service, you'll need the following:

- ENDPOINT -> This value can be found in the Keys & Endpoint section when examining your resource from the Azure portal. Alternatively, you can find the value in Azure OpenAI Studio > Playground > Code View. An example endpoint is: https://docs-test-001.openai.azure.com/.
- API-KEY -> This value can be found in the Keys & Endpoint section when examining your resource from the Azure portal. You can use either KEY1 or KEY2.
- DEPLOYMENT-NAME -> This value will correspond to the custom name you chose for your deployment when you deployed a model. This value can be found under Resource Management > Deployments in the Azure portal or alternatively under Management > Deployments in Azure OpenAI Studio.

Go to your resource in the Azure portal. The Endpoint and Keys can be found in the Resource Management section. Copy your endpoint and access key as you'll need both for authenticating your API calls. You can use either KEY1 or KEY2. Always having two keys allows you to securely rotate and regenerate keys without causing a service disruption.

#### Environment variables
Copy .env.sample to .env and fill in the values for your environment.

- OPENAI_API_KEY=
- OPENAI_API_BASE=


In [3]:
# Import Python libraries
import os
import openai
from Utilities.envVars import *

# Set OpenAI API key and endpoint
openai.api_type = "azure"
openai.api_version = OpenAiVersion
openai_api_key = OpenAiKey
assert openai_api_key, "ERROR: Azure OpenAI Key is missing"
openai.api_key = openai_api_key
openAiEndPoint = f"{OpenAiEndPoint}"
assert openAiEndPoint, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in openAiEndPoint.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = openAiEndPoint

#### Basic Completion

Let's look at a simple example to summarize a text. Remember that you can always control the output of a prompt via instructions, some typical controls are around:

- length of the summary
- target audience of the summary

In [7]:
# Give your prompt here
prompt = """Summarize this for a second-grade student:

        Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of 
        the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. 
        Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient
        civilizations since before recorded history. It is named after the Roman god Jupiter. 
        When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows, 
        and is on average the third-brightest natural object in the night sky after the Moon and Venus."""

messages=[
        {"role": "system", "content": prompt}
    ]

try:
    # Create a completion for the provided prompt and parameters
    # To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
    completion = openai.ChatCompletion.create(
                    messages=messages,
                    temperature=0,
                    max_tokens=500,
                    engine=OpenAiChat)

    # print the completion
    print(completion.choices[0].message.content.strip(" \n"))
    
    # Here indicating if the response is filtered
    if completion.choices[0].finish_reason == "content_filter":
        print("The generated content is filtered.")
        
except openai.error.APIError as e:
    # Handle API error here, e.g. retry or log
    print(f"OpenAI API returned an API Error: {e}")

except openai.error.AuthenticationError as e:
    # Handle Authentication error here, e.g. invalid API key
    print(f"OpenAI API returned an Authentication Error: {e}")

except openai.error.APIConnectionError as e:
    # Handle connection error here
    print(f"Failed to connect to OpenAI API: {e}")

except openai.error.InvalidRequestError as e:
    # Handle connection error here
    print(f"Invalid Request Error: {e}")

except openai.error.RateLimitError as e:
    # Handle rate limit error
    print(f"OpenAI API request exceeded rate limit: {e}")

except openai.error.ServiceUnavailableError as e:
    # Handle Service Unavailable error
    print(f"Service Unavailable: {e}")

except openai.error.Timeout as e:
    # Handle request timeout
    print(f"Request timed out: {e}")

Jupiter is a really big planet that is far away from the Sun. It is made mostly of gas and is much bigger than all the other planets combined. People have known about Jupiter for a really long time because it is very bright in the sky at night. It is named after a god from ancient Rome. Sometimes, Jupiter is so bright that it can make shadows on Earth. It is one of the brightest things we can see in the sky, after the Moon and Venus.


In [9]:
#Let's change the prompt to specify the length of the summary to be 3 sentences.
prompt = """Summarize this for a second-grade student in 3 sentences:

        Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of 
        the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. 
        Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient
        civilizations since before recorded history. It is named after the Roman god Jupiter. 
        When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows, 
        and is on average the third-brightest natural object in the night sky after the Moon and Venus."""

messages=[
        {"role": "system", "content": prompt}
    ]

# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.ChatCompletion.create(
                messages=messages,
                temperature=0,
                max_tokens=500,
                engine=OpenAiChat)

# print the completion
print(completion.choices[0].message.content.strip(" \n"))

Jupiter is a really big planet that is far away from the Sun. It is made of gas and is even bigger than all the other planets combined. People have known about Jupiter for a really long time because it is very bright in the sky at night. It is named after a god from ancient Rome.


In [10]:
#Let's now change the prompt to specify the target audience of the summary to be a physicist.
prompt = """Summarize this for a physicist in 3 sentences:

        Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of 
        the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. 
        Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient
        civilizations since before recorded history. It is named after the Roman god Jupiter. 
        When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows, 
        and is on average the third-brightest natural object in the night sky after the Moon and Venus."""

messages=[
        {"role": "system", "content": prompt}
    ]

# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.ChatCompletion.create(
                messages=messages,
                temperature=0,
                max_tokens=500,
                engine=OpenAiChat)


# print the completion
print(completion.choices[0].message.content.strip(" \n"))

Jupiter is the largest planet in our Solar System and is made mostly of gas. It has a mass that is 1/1000th that of the Sun, but is still 2.5 times more massive than all the other planets combined. It is very bright and can even cast visible shadows on Earth.


In [11]:
#You can easily control the format of the summarization as well to fit into desired formats. 
#Let's summarize the same text in a bullet point format.
prompt = """Summarize this for a second-grade student in 3 sentences in bullet points:

        Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of 
        the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. 
        Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient
        civilizations since before recorded history. It is named after the Roman god Jupiter. 
        When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows, 
        and is on average the third-brightest natural object in the night sky after the Moon and Venus."""

messages=[
        {"role": "system", "content": prompt}
    ]

# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.ChatCompletion.create(
                messages=messages,
                temperature=0,
                max_tokens=500,
                engine=OpenAiChat)

# print the completion
print(completion.choices[0].message.content.strip(" \n"))

- Jupiter is a really big planet that is the fifth one from the Sun.
- It is made mostly of gas and is way bigger than all the other planets put together.
- People have known about Jupiter for a really long time because it is really bright in the night sky.


In [12]:
#Summarize this for a second-grade student as a title and a one sentence abstract:
prompt = """Summarize this for a second-grade student as a title and a one sentence abstract:

        Jupiter is the fifth planet from the Sun and the largest in the Solar System. It is a gas giant with a mass one-thousandth that of 
        the Sun, but two-and-a-half times that of all the other planets in the Solar System combined. 
        Jupiter is one of the brightest objects visible to the naked eye in the night sky, and has been known to ancient
        civilizations since before recorded history. It is named after the Roman god Jupiter. 
        When viewed from Earth, Jupiter can be bright enough for its reflected light to cast visible shadows, 
        and is on average the third-brightest natural object in the night sky after the Moon and Venus."""

messages=[
        {"role": "system", "content": prompt}
    ]

# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.ChatCompletion.create(
                messages=messages,
                temperature=0,
                max_tokens=500,
                engine=OpenAiChat)

# print the completion
print(completion.choices[0].message.content.strip(" \n"))

Title: "Jupiter: The Biggest and Brightest Planet"

Abstract: Jupiter is a really big planet that is the fifth one from the Sun. It is so bright that we can see it in the night sky, and it is named after a Roman god.


In [13]:
#Sentiment Analysis
prompt = """Decide whether the following customer feedback is positive or negative.

        Q: I was disappointed with the quality of the product. It was very cheaply made and did not meet my expectations at all.
        Q: I was happy with this product, it is well made and great quality for the price.
        """

messages=[
        {"role": "system", "content": prompt}
    ]

# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.ChatCompletion.create(
                messages=messages,
                temperature=0,
                max_tokens=500,
                engine=OpenAiChat)

# print the completion
print(completion.choices[0].message.content.strip(" \n"))

The first customer feedback is negative. The second customer feedback is positive.


In [14]:
#PII Detection
prompt =  """List all PII data from following statement:
        John Doe is a 35-year old man and he lives at 21 Main Street, New York, NY. He is a software engineer and he works at Google. He has a wife named Jane Doe and they have two children
        """

messages=[
        {"role": "system", "content": prompt}
    ]

# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.ChatCompletion.create(
                messages=messages,
                temperature=0,
                max_tokens=500,
                engine=OpenAiChat)


# print the completion
print(completion.choices[0].message.content.strip(" \n"))

- John Doe (name)
- 35 (age)
- 21 Main Street, New York, NY (address)
- Software engineer (occupation)
- Google (employer)
- Jane Doe (wife's name)
- Two children (family information)


In [15]:
#Code Generation
prompt =  """### Postgres SQL tables, with their properties:
        #
        # Employee(id, name, department_id)
        # Department(id, name, address)
        # Salary_Payments(id, employee_id, amount, date)
        #
        ### A query to list the names of the departments which employed more than 10 employees in the last 3 months
        query: SELECT"""

messages=[
        {"role": "system", "content": prompt}
    ]

# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.ChatCompletion.create(
                messages=messages,
                temperature=0,
                max_tokens=500,
                engine=OpenAiChat)

# print the completion
print(completion.choices[0].message.content.strip(" \n"))

Department.name
        FROM Department
        INNER JOIN Employee ON Department.id = Employee.department_id
        WHERE Employee.id IN (
            SELECT employee_id
            FROM Salary_Payments
            WHERE date >= CURRENT_DATE - INTERVAL '3 months'
            GROUP BY employee_id
            HAVING COUNT(*) > 10
        )


In [16]:
#Code Explanation
prompt =  """Can you explain what does this code do?
        #
        # ###
        Code:
        SELECT d.name FROM Department d JOIN Employee e ON d.id = e.department_id WHERE e.id IN (SELECT employee_id 
        FROM Salary_Payments WHERE date > now() - interval '3 months') GROUP BY d.name HAVING COUNT(*) > 10
        #
        #
        Answer:
        #"""

messages=[
        {"role": "system", "content": prompt}
    ]

# Create a completion for the provided prompt and parameters
# To know more about the parameters, checkout this documentation: https://learn.microsoft.com/en-us/azure/cognitive-services/openai/reference
completion = openai.ChatCompletion.create(
                messages=messages,
                temperature=0,
                max_tokens=500,
                engine=OpenAiChat)

# print the completion
print(completion.choices[0].message.content.strip(" \n"))

This code is a SQL query that retrieves the names of departments that have more than 10 employees who have received salary payments in the last 3 months.

The query starts by selecting the name column from the Department table. It then joins the Employee table on the condition that the department_id in the Employee table matches the id in the Department table.

Next, it uses a subquery to select the employee_id from the Salary_Payments table where the date is greater than the current date minus 3 months.

The main query then groups the results by the department name and applies a filter using the HAVING clause. It only includes departments where the count of employees is greater than 10.

In summary, this query retrieves the names of departments that have more than 10 employees who have received salary payments in the last 3 months.


In [20]:
# Helper Function to call OpenAI
def callOpenAi(num_times, messages, temperature):
    for i in range(num_times):
        response = openai.ChatCompletion.create(
            engine=OpenAiChat,
            messages=messages,
            max_tokens=60,
            temperature = temperature
        )
        print(response.choices[0].message.content.strip(" \n"))

In [23]:
# Temperature - Defaults to 1
# What sampling temperature to use. Higher values means the model will take more risks. 
# Try 0.9 for more creative applications, and 0 (argmax sampling) for ones with a well-defined answer.
#We generally recommend altering this or top_p but not both.
messages=[
        {"role": "system", "content": 'The best pet is a '}
    ]
callOpenAi(10, messages, temperature = 0)

subjective question and can vary depending on individual preferences and circumstances. Some people may prefer dogs as pets because they are loyal, affectionate, and can be trained to perform various tasks. Others may prefer cats because they are independent, low-maintenance, and provide companionship. Some people may prefer smaller
subjective question and can vary depending on individual preferences and circumstances. Some people may prefer dogs as pets because they are loyal, affectionate, and can be trained to perform various tasks. Others may prefer cats because they are independent, low-maintenance, and provide companionship. Some people may prefer smaller
subjective question and can vary depending on individual preferences and circumstances. Some people may prefer dogs as pets because they are loyal, affectionate, and can be trained to perform various tasks. Others may prefer cats because they are independent, low-maintenance, and provide companionship. Some people may prefer sma

In [24]:
# Let's vary the temperature and set it to "1"
messages=[
        {"role": "system", "content": 'The best pet is a '}
    ]
callOpenAi(10, messages, temperature = 1)

subjective choice and varies from person to person. Some may prefer dogs due to their loyalty and companionship, while others may prefer cats for their independence and low-maintenance nature. Others may prefer smaller pets like rabbits, hamsters, or guinea pigs, while some may even prefer reptiles or
subjective topic and can vary depending on individual preferences and lifestyles. However, some popular choices for pet animals include dogs, cats, and fish.

Dogs are often considered to be the best pets because of their loyalty, affectionate nature, and ability to provide companionship. They are also known for
subjective choice and depends on an individual's personal preferences and lifestyle. Some people may argue that dogs make the best pets because they are loyal, affectionate, and can be trained to perform various tasks. Others may argue that cats make the best pets because they are more independent, low maintenance, and
matter of personal preference, as different pets have differen

In [25]:
# Helper Function to call OpenAI
def callOpenAiTopP(num_times, messages, top_p):
    for i in range(num_times):
        response = openai.ChatCompletion.create(
            engine=OpenAiChat,
            messages=messages,
            max_tokens=60,
            top_p = top_p
        )
        print(response.choices[0].message.content.strip(" \n"))

In [27]:
# top_p - Defaults to 1
# An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens 
# with top_p probability mass. So 0.1 means only the tokens comprising the top 10% probability mass are considered.
# We generally recommend altering this or temperature but not both.
messages=[
        {"role": "system", "content": 'The best pet is a '}
    ]
callOpenAiTopP(5, messages, top_p = 1)

subjective choice and may vary depending on individual preferences, lifestyle, and circumstances. Some people may prefer dogs as they are loyal, trainable, and provide companionship. Others may prefer cats for their independent nature and low maintenance. Some people may even prefer exotic pets such as birds, reptiles, or
subjective matter and can vary depending on individual preferences and circumstances. Some people may prefer dogs as pets because they are loyal, loving, and can be trained to perform various tasks and tricks. Others may prefer cats because they are independent, low-maintenance, and provide comfort and companionship. Other popular
subjective topic and can vary depending on individual preferences and circumstances. Some people may prefer dogs because of their loyalty, companionship, and ability to be trained. Others may prefer cats because of their independence and low maintenance. Some may prefer smaller animals like rabbits, guinea pigs, or hamsters because
matter of

In [28]:
# Let's vary the top_p and set it to "0.1"
messages=[
        {"role": "system", "content": 'The best pet is a '}
    ]
callOpenAiTopP(5, messages, top_p = 0.1)

subjective question and can vary depending on individual preferences and circumstances. Some people may prefer dogs as pets because they are loyal, affectionate, and can be trained to perform various tasks. Others may prefer cats because they are independent, low-maintenance, and provide companionship. Some people may prefer smaller
subjective question and can vary depending on individual preferences and circumstances. Some people may prefer dogs as pets because they are loyal, affectionate, and can be trained to perform various tasks. Others may prefer cats because they are independent, low-maintenance, and provide companionship. Some people may prefer smaller
subjective question and can vary depending on individual preferences and circumstances. Some people may prefer dogs as pets because they are loyal, affectionate, and can be trained to perform various tasks. Others may prefer cats because they are independent, low-maintenance, and provide companionship. Some people may prefer sma